In [14]:
from openfisca_us import Microsimulation
from openfisca_us.api import *
from openfisca_us.entities import *
from openfisca_us.tools.general import *
from openfisca_us import reforms
import numpy as np

In [15]:
# Calculate poverty rate
base = Microsimulation(year=2020)
base.calc("in_poverty", map_to="person").mean()

0.10159000907591269

In [16]:
class basic_income(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "Basic income"

    def formula(tax_unit, period, parameters):
        # Extract FPG parameter
        fpg_params = parameters(period).poverty.fpg
        # Includes first_person and additional_person
        # Extract tax unit number of people
        nb_people = tax_unit.nb_persons()
        # Calculate FPG
        fpg = fpg_params.first_person.contiguous_US + (nb_people - 1) * fpg_params.additional_person.contiguous_US
        # Extract taxable income
        taxable_income = tax_unit("taxable_income", period)
        # Calculate basic income phased out at 50%
        return np.maximum(fpg - taxable_income * 0.5, 0)
        # TODO: Make 50% a parameter

In [17]:
class taxable_income(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR

In [18]:
class SPM_unit_net_income(Variable):
    value_type = float
    entity = SPMUnit
    label = u"SPM unit net income"
    definition_period = YEAR

    def formula(spm_unit, period):
        INCOME_COMPONENTS = [
            "SPM_unit_total_income",
            "SPM_unit_SNAP",
            "SPM_unit_capped_housing_subsidy",
            "SPM_unit_school_lunch_subsidy",
            "SPM_unit_energy_subsidy",
            "SPM_unit_WIC",
        ]
        EXPENSE_COMPONENTS = [
            "SPM_unit_FICA",
            "SPM_unit_federal_tax",
            "SPM_unit_state_tax",
            "SPM_unit_capped_work_childcare_expenses",
            "SPM_unit_medical_expenses",
        ]
        income = add(spm_unit, period, *INCOME_COMPONENTS)
        expense = add(spm_unit, period, *EXPENSE_COMPONENTS)
        basic_income = sum_contained_tax_units("basic_income", spm_unit, period)
        return income - expense + basic_income

In [19]:
class reform(Reform):
    def apply(self):
        self.update_variable(SPM_unit_net_income)
        self.add_variable(basic_income)
        self.add_variable(taxable_income)

# reform = reforms.restructure(SPM_unit_net_income), reforms.new_variable(basic_income)
reform_sim = Microsimulation(reform, year=2020)
from openfisca_us_data import RawCPS
taxable_income_values = RawCPS.load(2020, "tax_unit").TAX_INC.values
reform_sim.simulation.set_input("taxable_income", 2020, taxable_income_values)

In [20]:
import pandas as pd
df = pd.DataFrame(dict(
    net_income = reform_sim.calc("SPM_unit_net_income", map_to="spm_unit"),
    total_income = reform_sim.calc("SPM_unit_total_income", map_to="spm_unit"),
    taxable_income = reform_sim.calc("taxable_income", map_to="spm_unit"),
    basic_income = reform_sim.calc("basic_income", map_to="spm_unit")))
df.corr()

,net_income,total_income,taxable_income,basic_income
net_income,1.000000,0.976896,-0.026355,0.252275
total_income,0.976896,1.000000,0.014944,0.101657
taxable_income,-0.026355,0.014944,1.000000,-0.350393
basic_income,0.252275,0.101657,-0.350393,1.000000


In [21]:
gain = reform_sim.calc("SPM_unit_net_income") - base.calc("SPM_unit_net_income")
gain.groupby(base.calc("SPM_unit_net_income").decile_rank()).mean()

1.0      6950.139729
2.0      7377.274381
3.0      8676.671452
4.0      9226.481163
5.0      9980.518635
6.0     10343.661848
7.0     11180.844588
8.0     11710.768067
9.0     12154.836279
10.0    12263.114358
dtype: float64

In [22]:
reform_sim.calc("SPM_unit_net_income").mean() - base.calc("SPM_unit_net_income").mean()

9986.52237030775

In [23]:
reform_sim.calc("basic_income").mean()

7837.700207688253

In [27]:
reform_income = pd.Series(reform_sim.calc("SPM_unit_net_income").values, index=reform_sim.calc("spm_unit_id").values)

In [43]:
SPM_unit_UBI = reform_sim.calc("basic_income", map_to="person")

In [52]:
spm_unit_size = reform_sim.simulation.populations["spm_unit"].project(
    reform_sim.simulation.populations["spm_unit"].nb_persons()
)

In [54]:
spm_unit_size.size

157959

In [55]:
SPM_unit_UBI /= spm_unit_size

In [57]:
SPM_unit_UBI.groupby(reform_sim.calc("person_spm_unit_id").values).sum()

KeyboardInterrupt: 

In [71]:
basic_income = reform_sim.map_to(reform_sim.calc("basic_income", map_to="person", how="mean"), "person", "spm_unit")

In [73]:
taxable_income = reform_sim.map_to(reform_sim.calc("taxable_income", map_to="person", how="mean"), "person", "spm_unit")

In [75]:
pd.concat([
    pd.DataFrame(dict(basic_income=basic_income, taxable_income=taxable_income)),
    reform_sim.df(["spm_unit_id", "in_poverty"]),
], axis=1)

,basic_income,taxable_income,spm_unit_id,in_poverty
0,0.0,98944.0,1001.0,False
1,4880.0,24720.0,2001.0,False
2,12760.0,0.0,3001.0,False
3,18225.5,14589.0,4001.0,False
4,42760.0,0.0,5001.0,False
...,...,...,...,...
63087,12760.0,0.0,91497000.0,False
63088,0.0,37600.0,91498008.0,False
63089,10310.0,4900.0,91498008.0,True
63090,0.0,28700.0,91499000.0,False


In [70]:
reform_sim.calc("spm_unit_id")

            value       weight
0          1001.0  1552.900024
1          2001.0   990.489990
2          3001.0  1505.270020
3          4001.0  1430.699951
4          5001.0  1133.369995
...           ...          ...
63087  91497000.0   486.019989
63088  91498008.0   444.179993
63089  91498008.0   388.290009
63090  91499000.0   494.049988
63091  91500000.0   423.839996

[63092 rows x 2 columns]